# Project 1: Fire Extinguisher

James Burns & Remington Katz

CS 520

In [14]:
import random
import copy

### 1 The Ship

In [27]:
class ship():
    def __init__(self, d):
        self.dim = d
        self.ship_display = [['■' for i in range(d)] for j in range(d)] #■ for blocked i.e. not open
        self.start = ()

        #Choose a square in the interior to open at random
        y = random.randint(1, d-2)
        x = random.randint(1, d-2)
        self.ship_display[y][x] = 'O' #O for open
        #The board display can be imagined as Quadrant IV of a cartesian coordinate system, where the origin is in the top left and you read a coordinate as y spaces down and x spaces to the right
        #Iteratively do the following... (from assignment)
        while True:
            one_open_neighbor = list()
            for row in range(d):
                for col in range(d):
                    if self.ship_display[row][col] == '■':
                        open_neighbor_count = 0
                        if (row+1 <= d-1) and (self.ship_display[row+1][col] == 'O'):
                            open_neighbor_count += 1
                        if (row-1 >= 0) and (self.ship_display[row-1][col] == 'O'):
                            open_neighbor_count += 1
                        if (col+1 <= d-1) and (self.ship_display[row][col+1] == 'O'):
                            open_neighbor_count += 1
                        if (col-1 >= 0) and (self.ship_display[row][col-1] == 'O'):
                            open_neighbor_count += 1
                        if open_neighbor_count == 1:
                            one_open_neighbor.append((row,col))
            if len(one_open_neighbor) == 0:
                break
            random_neighbor = random.sample(one_open_neighbor, 1)[0]
            self.ship_display[random_neighbor[0]][random_neighbor[1]] = 'O'
        #ship.print_ship(self)
        
        #Open dead ends; pick random closed neighbor and open it
        dead_ends_list = list()
        for row in range(d):
            for col in range(d):
                if self.ship_display[row][col] == 'O':
                    open_neighbor_count = 0
                    if (row+1 <= d-1) and (self.ship_display[row+1][col] == 'O'):
                        open_neighbor_count += 1
                    if (row-1 >= 0) and (self.ship_display[row-1][col] == 'O'):
                        open_neighbor_count += 1
                    if (col+1 <= d-1) and (self.ship_display[row][col+1] == 'O'):
                        open_neighbor_count += 1
                    if (col-1 >= 0) and (self.ship_display[row][col-1] == 'O'):
                        open_neighbor_count += 1
                    if open_neighbor_count == 1:
                        dead_ends_list.append((row,col))
        half = len(dead_ends_list)//2
        dead_ends = random.sample(dead_ends_list, half)
        #print(dead_ends_list)
        #print(dead_ends)
        for dead_end in dead_ends:
            dead_end_blocked_neighbor = list()
            if (dead_end[0]+1 <= d-1) and (self.ship_display[dead_end[0]+1][dead_end[1]] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0]+1, dead_end[1]))
            if (dead_end[0]-1 >= 0) and (self.ship_display[dead_end[0]-1][dead_end[1]] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0]-1, dead_end[1]))
            if (dead_end[1]+1 <= d-1) and (self.ship_display[dead_end[0]][dead_end[1]+1] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0], dead_end[1]+1))
            if (dead_end[1]-1 >= 0) and (self.ship_display[dead_end[0]][dead_end[1]-1] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0], dead_end[1]-1))
            #print(dead_end_blocked_neighbor)
            if dead_end_blocked_neighbor != []: #Make sure block neighbor was not previosuly removed
                random_blocked_neighbor = random.sample(dead_end_blocked_neighbor, 1)[0]
                self.ship_display[random_blocked_neighbor[0]][random_blocked_neighbor[1]] = 'O'
        
        self.open_ship = copy.deepcopy(self.ship_display) #Keep a copy of the original ship to see which cells are opened/blocked
    
    
    def print_open_ship(self):
        for row in self.open_ship:
            formatted_row = ["{:1}".format(x) for x in row]
            print(" ".join(formatted_row))

    def flatten(self, row, col):
	    return row*self.dim + col



In [16]:
def print_ship(ship):
    for row in ship.ship_display:
        formatted_row = ["{:1}".format(x) for x in row]
        print(" ".join(formatted_row))

In [17]:
ship_1 = ship(10)
ship_1.print_open_ship()
print()
print_ship(ship_1)

O O O O O O O ■ O O
O O ■ O ■ ■ O ■ O O
■ O O O ■ O O O O ■
O O ■ ■ O O ■ ■ O O
■ O ■ ■ O O O O O O
O O O O ■ O ■ O O O
O O O O O O O O O O
O O O O ■ ■ O ■ O ■
O O O O O O O ■ ■ O
■ O ■ O O ■ O O O O

O O O O O O O ■ O O
O O ■ O ■ ■ O ■ O O
■ O O O ■ O O O O ■
O O ■ ■ O O ■ ■ O O
■ O ■ ■ O O O O O O
O O O O ■ O ■ O O O
O O O O O O O O O O
O O O O ■ ■ O ■ O ■
O O O O O O O ■ ■ O
■ O ■ O O ■ O O O O


### 3 The Fire

In [70]:
def prob_catching_fire(q, K):
    return 1 - (1-q)**K

def place_element(ship, element):
    random_cell_state = None
    while random_cell_state != 'O':
        row = random.randint(0, ship.dim-1)
        col = random.randint(0, ship.dim-1)
        if ship.open_ship[row][col] == 'O':
            ship.ship_display[row][col] = element #F for fire
            random_cell_state = 'O'
            if element == 'B':
                ship.start = (row,col)
    return ship

In [19]:
ship_1 = place_element(ship_1, 'F')

In [20]:
def spread_fire(timestep, ship, q):
    for t in range(timestep):
        ship_copy = copy.deepcopy(ship)
        for row in range(ship.dim):
            for col in range(ship.dim):
                if (ship.open_ship[row][col] == 'O') and (ship.ship_display[row][col] != 'F'):
                    neighbors_on_fire = 0
                    if (row+1 <= ship.dim-1) and (ship.ship_display[row+1][col] == 'F'):
                        neighbors_on_fire += 1
                    if (row-1 >= 0) and (ship.ship_display[row-1][col] == 'F'):
                        neighbors_on_fire += 1
                    if (col+1 <= ship.dim-1) and (ship.ship_display[row][col+1] == 'F'):
                        neighbors_on_fire += 1
                    if (col-1 >= 0) and (ship.ship_display[row][col-1] == 'F'):
                        neighbors_on_fire += 1
                    if random.uniform(0, 1) < prob_catching_fire(q, neighbors_on_fire):
                        ship_copy.ship_display[row][col] = 'F'
        ship = ship_copy
        # print_ship(ship)
        # print()

    return ship

In [21]:
ship_1 = spread_fire(10, ship_1, 0.5)
print_ship(ship_1)

F F F F F F F ■ O O
F F ■ F ■ ■ F ■ O O
■ F F F ■ O F O O ■
F F ■ ■ O O ■ ■ O O
■ F ■ ■ O O O O O O
O F O O ■ O ■ O O O
O O O O O O O O O O
O O O O ■ ■ O ■ O ■
O O O O O O O ■ ■ O
■ O ■ O O ■ O O O O


5 The Task

In [520]:
def bot1(ship):
	neighbors = []
	visited = set()
	prev = dict()
	neighbors.append(ship.start)
	visited.add(ship.start)
	prev[ship.start] = None
	board = ship.ship_display
	found = False
	while(True):
		if not neighbors:
			break
		row, col = neighbors.pop(0)
		if(ship.ship_display[row][col] == 'G'):
			return retrace(prev, (row, col))
			break
		
		if(row - 1 >= 0 and ship.ship_display[row-1][col] != '■' and ship.ship_display[row-1][col] != 'F' and (row-1, col) not in visited):
		   neighbors.append((row-1, col))
		   visited.add((row-1, col))
		   prev[(row-1, col)] = (row, col)
		
		if(row + 1 < ship.dim and ship.ship_display[row+1][col] != '■' and ship.ship_display[row+1][col] != 'F' and (row+1, col) not in visited):
		   neighbors.append((row+1, col))
		   visited.add((row+1, col))
		   prev[(row+1, col)] = (row, col)

		if(col-1 >= 0 and ship.ship_display[row][col-1] != '■' and ship.ship_display[row][col-1] != 'F' and (row, col-1) not in visited):
		   neighbors.append((row, col-1))
		   visited.add((row, col-1))
		   prev[(row, col-1)] = (row, col)

		if(col + 1 < ship.dim and ship.ship_display[row][col+1] != '■' and ship.ship_display[row][col+1] != 'F' and (row, col+1) not in visited):
		   neighbors.append((row, col+1))
		   visited.add((row, col+1))
		   prev[(row, col+1)] = (row, col)

def retrace(prev, start):
	route = []
	while(prev[start] != None):
		route.append(start)
		start = prev[start]
	route.reverse()	
	return route


In [515]:
d = 20
ship_2 = ship(d)

place_element(ship_2,'F')
place_element(ship_2,'B')
place_element(ship_2,'G')

print(ship_2.start)
print_ship(ship_2)
bot1(ship_2)

(13, 9)
O O O O O O ■ O O O O O F ■ O O O O O O
O ■ O O O O ■ O O ■ O ■ O O O ■ O O ■ O
O ■ O ■ O O O O O ■ O ■ O ■ O O O O O O
O O O O ■ O ■ ■ O O O O O O ■ ■ O O O ■
■ O ■ O ■ O O ■ O ■ O ■ O O O O O ■ ■ O
O O O O O O ■ O O ■ O O ■ ■ ■ O O ■ O O
O O O O O ■ O O O O O ■ O ■ ■ O ■ O ■ O
O ■ O ■ G O O O O ■ O O O O O O O O ■ O
O O O O ■ O O O ■ O O ■ O ■ O ■ ■ O O O
■ O ■ O O O O O O O ■ ■ O ■ O ■ O O ■ O
■ O O ■ O O ■ O ■ O O O O O O O O O O O
■ ■ O O ■ O ■ O O O ■ O O O O ■ O ■ ■ ■
■ O ■ O O O O O ■ O O O ■ ■ O O O O O O
O O O O ■ O O O ■ B ■ O ■ ■ ■ O ■ O ■ O
O O ■ O ■ O ■ ■ O O ■ O O ■ O O O O ■ O
O O O O O O O ■ ■ O ■ ■ O ■ O ■ O O O ■
O ■ O ■ ■ O O O O O O O O O O ■ O ■ O O
O O O O ■ O ■ O O O ■ O ■ O ■ O O O ■ O
O ■ ■ O O ■ ■ ■ O O ■ O O O O O O ■ O O
O O O O ■ ■ ■ ■ ■ O O ■ O O O O O O O ■
Found


[(12, 9),
 (11, 9),
 (10, 9),
 (9, 9),
 (9, 8),
 (9, 7),
 (8, 7),
 (7, 7),
 (7, 6),
 (7, 5),
 (7, 4)]

In [525]:
burnCount = 0
runs = 100
for k in range(runs):
    shipTest = ship(30)
    place_element(shipTest,'F')
    place_element(shipTest,'B')
    place_element(shipTest,'G')
    route = bot1(shipTest)
    if(not route): 
        burnCount += 1
    else:
        for i in range(len(route)):
            row, col = route[i]
            if shipTest.ship_display[row][col] == 'F':
                burnCount += 1
                break
            shipTest = spread_fire(1, shipTest, 0.5)

runs-burnCount

74

In [533]:
burnCount = 0
runs = 100
for k in range(runs):
    shipTest = ship(30)
    place_element(shipTest,'F')
    place_element(shipTest,'B')
    place_element(shipTest,'G')
    stop = False
    while(not stop): 
        route = bot1(shipTest)
        if(not route):
            burnCount += 1
            break
        else:
            row, col = route[0]
            shipTest.start = (row,col)
            if(shipTest.ship_display[row][col] == 'G'):
                break
            shipTest = spread_fire(1, shipTest, 0.5)
runs-burnCount

69